## Initial work
Modified the part that sets the folder paths, etc. (changed to the methods used in the PSR_MVPA_crossvalidation_fix.m file from the included .zip).

In [ ]:
% Try-out to do time-generalized cross-decoding more clean with native
% Cosmo functions
% UPDATE: Try to parallelize decoding

%% Init                         
% initialize
clearvars; close all; clc;

% add paths to functions
PSR_setpaths;

% set folders
folder.IN   = fullfile(cd,'TIMELOCKED','S1');
folder.TRL  = fullfile(cd,'TRIALS','REJECTVISUAL');
folder.INFO = fullfile(cd,'TRIALS','INFO');
folder.OUT  = cd;

% subject
subjects = PSR_subjects(folder.IN);
subjects = subjects(2:end);

% names for suffices of different data subsets
suffix = {'LEFTS1','RIGHTS1','SACS1','SACON','SACOFF'};

## Parameters
Below, the parameters are given, with their default values. This cell is tagged with the tagg "parameters". Papermill would inject a cell with parameters below this one, overriding the defaults with whatever setting you want.

In [ ]:
%% Parameters                   
p.dsfs         = 250;         % downsampled frequency --> was 250
p.bws          = -0.2;        % baseline window start
p.bwe          = 0;           % baseline window end
p.mintime      = -0.1;        % do not use the whole eocht from -0.5 to 1.5, but start at [mintime]
p.maxtime      = 0.9;         % do not use the whole eocht from -0.5 to 1.5, but end at [maxtime]
p.saccut       = [-0.6 0.4];  % saccade epoch
p.sactimeshift = p.maxtime - max(p.saccut); % time values have to be equal for time-generalization

## Core code
Left unchanged. Suggestion: consider throwing an error (or at least printing some message) if the run is aborted because the desired output file already exists.

In [ ]:
% CoSMo measure argurments
p.measure                 = @cosmo_dim_generalization_measure;
p.measure_args            = struct();
p.measure_args.measure    = @cosmo_crossvalidation_measure;
p.measure_args.classifier = @cosmo_classify_svm;
p.measure_args.dimension  = 'time';
p.measure_args.radius     = 2;
p.measure_args.output     = 'accuracy';        

%% Loop over subjects           
for isubj = 1:numel(subjects)
    %% Check if already processed           
    files = dir(folder.OUT);
    files = cat(1,{files.name});
    
    % we should have 6 files for each subject
    if sum(contains(files,subjects{isubj})) == 6
        continue;
    end
    
    %% Load dataset                         
    fprintf('Start: %s, %s\n',subjects{isubj}, datestr(now,31));
    fprintf('Loading dataset... ');
    tic;
    
    [data] = PSR_loadTimelock(folder.IN,folder.TRL,'S1',subjects{isubj},'MEG');
    dataS1 = data.s1lock;
    dataS1 = rmfield(dataS1, 'cfg');
    dataS1 = ft_struct2double(dataS1);
    
    clearvars data;
    fprintf('done! Time elapsed: %3.1f\n',toc);
    
    %% Downsample                           
    tic;
    fprintf('Downsampling data to %d Hz... ',p.dsfs);
    
    dataS1.fsample = 500;
    cfg            = [];
    cfg.resamplefs = p.dsfs;
    dataS1         = ft_resampledata(cfg,dataS1);
    dataS1.time    = cellfun(@(x) round(x*1000)/1000, dataS1.time, 'UniformOutput', false);
    
    fprintf('done! Time elapsed: %3.1f\n',toc);
    
    %% Standardize data                     
    tic;
    fprintf('Standardizing data... ');
    
    % 1. baseline correction
    cfg = [];
    cfg.demean = 'yes';
    cfg.baselinewindow = [p.bws p.bwe];
    [dataS1] = ft_preprocessing(cfg,dataS1);
    
    % 2. standardize data per sensor
    i1 = find( abs(dataS1.time{1}-p.bws) == min(abs(dataS1.time{1}-p.bws)) );
    i2 = find( abs(dataS1.time{1}-p.bwe) == min(abs(dataS1.time{1}-p.bwe)) );
    bw_std = std( cell2mat( cellfun(@(x) x(:,i1:i2),dataS1.trial,'UniformOutput',false) ), [], 2);
    dataS1.trial = cellfun(@(x) bsxfun(@rdivide,x,bw_std), dataS1.trial,'UniformOutput',false);
    
    fprintf('done! Time elapsed: %3.1f\n',toc);
    
    %% CoSMo data: S1 onset aligned         
    tic;
    fprintf('\nPreparing CoSMO data, aligned to S1...\n');
    for k = find(contains(suffix,'S1'))
        % 1. select conditions
        cfg = [];
        switch k
            case find(contains(suffix,'LEFTS1'))
                cfg.trials = PSR_selectCondition(dataS1,{'fll','flh'});
            case find(contains(suffix,'RIGHTS1'))
                cfg.trials = PSR_selectCondition(dataS1,{'frl','frh'});
            case find(contains(suffix,'SACS1'))
                cfg.trials = PSR_selectCondition(dataS1,{'srl','srh'});
        end
        [dataTMP] = ft_selectdata(cfg,dataS1);
        
        % 2. make CoSMoMVPA data
        ds.(suffix{k}) = PSR_mkCoSMoData(dataTMP);
        clear dataTMP
        
        % 3. make epoch smaller (from [-0.5,1.5] to [mintime,maxtime])
        i1 = find(ds.(suffix{k}).a.fdim.values{2}==p.mintime) - p.measure_args.radius;
        i2 = find(ds.(suffix{k}).a.fdim.values{2}==p.maxtime) + p.measure_args.radius;
        timemask = ds.(suffix{k}).fa.time >=i1 & ds.(suffix{k}).fa.time <= i2;
        ds.(suffix{k}) = cosmo_slice(ds.(suffix{k}), timemask, 2);
        ds.(suffix{k}) = cosmo_dim_prune(ds.(suffix{k}));
        cosmo_check_dataset(ds.(suffix{k}));
        
        % 4. balance trial types
        ds.(suffix{k}) = PSR_balanceCoSMoTrialtype(ds.(suffix{k}), 'sfori');
        cosmo_check_dataset(ds.(suffix{k}));
        
    end
    fprintf('Done! Time elapsed: %3.1f\n',toc);
    
    %% CoSMo data: saccade onset aligned    
    tic;
    fprintf('\nPreparing CoSMO data, aligned to saccade onset...\n');
    
    % index of 'saccade onset' suffix
    isfx = find(contains(suffix,'SACON'));
    
    % 1. SACCADE data: cut and align epochs to saccade onset
    [dataTMP] = PSR_cutsaconset( folder.INFO, dataS1, subjects{isubj}, 'SAC', ...
        p.saccut+[-p.measure_args.radius/p.dsfs p.measure_args.radius/p.dsfs]);
    
    % 2. select conditions for SACCADE data
    cfg            = [];
    cfg.trials     = PSR_selectCondition(dataTMP,{'srl','srh'});
    dataTMP        = ft_selectdata(cfg,dataTMP);
    ds.(suffix{isfx}) = PSR_mkCoSMoData(dataTMP);
    clear dataTMP

    % 3. balance trial types
    ds.(suffix{isfx}) = PSR_balanceCoSMoTrialtype(ds.(suffix{isfx}), 'sfori');
    
    % 4. small hack: shift time values in saccade onset data to match the time
    %    values in stimulus onset data
    ds.(suffix{isfx}).a.fdim.values{2} = ds.SACS1.a.fdim.values{2};
    
    % 5. verify that dataset is correct according to CoSMo guidelines
    cosmo_check_dataset(ds.(suffix{isfx}));
    fprintf('Done! Time elapsed: %3.1f\n',toc);
    
    %% CoSMo data: saccade offset aligned   
    tic;
    fprintf('\nPreparing CoSMO data, aligned to saccade offset...\n');
    
    % index of 'saccade onset' suffix
    isfx = find(contains(suffix,'SACOFF'));
    
    % 1. SACCADE data: cut and align epochs to saccade onset
    [dataTMP] = PSR_cutsaconset( folder.INFO, dataS1, subjects{isubj}, 'SACOFF', ...
        p.saccut+[-p.measure_args.radius/p.dsfs p.measure_args.radius/p.dsfs]);
    
    % 2. select conditions for SACCADE data
    cfg            = [];
    cfg.trials     = PSR_selectCondition(dataTMP,{'srl','srh'});
    dataTMP        = ft_selectdata(cfg,dataTMP);
    ds.(suffix{isfx}) = PSR_mkCoSMoData(dataTMP);
    clear dataTMP

    % 3. balance trial types
    ds.(suffix{isfx}) = PSR_balanceCoSMoTrialtype(ds.(suffix{isfx}), 'sfori');
    
    % 4. small hack: shift time values in saccade onset data to match the time
    %    values in stimulus onset data
    ds.(suffix{isfx}).a.fdim.values{2} = ds.SACS1.a.fdim.values{2};
    
    % 5. verify that dataset is correct according to CoSMo guidelines
    cosmo_check_dataset(ds.(suffix{isfx}));
    clearvars dataS1 dataTMP
    fprintf('Done! Time elapsed: %3.1f\n',toc);
    
    %% CoSMo data: prepare data for parfor  
    dsTMP = cell(1,6);
    fileappend = cell(1,6);
    for k = 1:6
        
        % select TRAIN and TEST data
        switch k 
            case 1     
                dsTRAIN = ds.RIGHTS1;
                dsTEST = ds.SACS1;
                fileappend{k} = 'rights1_sacs1';
            case 2
                dsTRAIN = ds.LEFTS1;
                dsTEST = ds.SACS1;
                fileappend{k} = 'lefts1_sacs1';
            case 3
                dsTRAIN = ds.RIGHTS1;
                dsTEST = ds.SACON;
                fileappend{k} = 'rights1_sacon';
            case 4
                dsTRAIN = ds.LEFTS1;
                dsTEST = ds.SACON;
                fileappend{k} = 'lefts1_sacon';
            case 5
                dsTRAIN = ds.RIGHTS1;
                dsTEST = ds.SACOFF;
                fileappend{k} = 'rights1_sacoff';
            case 6
                dsTRAIN = ds.LEFTS1;
                dsTEST = ds.SACOFF;
                fileappend{k} = 'lefts1_sacoff';
        end
        
        % combine to make single data structure
        dsTMP{k} = cosmo_stack({dsTRAIN,dsTEST});
        
        % set targets
        dsTMP{k}.sa.targets = dsTMP{k}.sa.sf;
        
        % make 2 chunks: in cosmo_dim_generalization_measure chunk 1 is always used
        % for training, chunck 2 is always used for testing.
        dsTMP{k}.sa.chunks = dsTMP{k}.sa.cndn + 1;
        
        % transpose time to the sample dimension
        dsTMP{k} = cosmo_dim_transpose(dsTMP{k},'time',1);
        
    end
    
    % replicate parameter struct in cell
    pcell = repmat({p},[1 6]);
    
    % clear redundant variables
    clearvars -except ds dsTMP folder p pcell subjects suffix fileappend isubj
    
    %% MVPA                                 
    resultcell = cell(1,6);
    parfor k = 1:6
        % run time-generalized cross-decoding searchlight
        resultcell{k} = pcell{k}.measure(dsTMP{k}, pcell{k}.measure_args);
    end
    
    % save data
    for k = 1:6
        result  = resultcell{k};
        fileOUT = [folder.OUT filesep subjects{isubj} '_' fileappend{k} '.mat'];
        save(fileOUT,'p','result','-v7.3');
    end
    
    fprintf('Done: %s, %s\n\n',subjects{isubj}, datestr(now,31));
    
    % remove large, redundant variables
    clearvars ds fileappend resultcell result pcell

end